In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import scipy
from scipy.integrate import odeint

In [3]:
covid_stats=pd.read_csv("covid-statistics-by-us-states-daily-updates.csv")
covid_stats=covid_stats.drop(['Unnamed: 0', 'posneg', 'hash', 'commercialscore', 'negativeregularscore',
                             'negativescore', 'positivescore', 'grade', 'score','dataqualitygrade',
                              'lastupdateet', 'datemodified','checktimeet', 'datechecked', 'fips', 'hash',
                              'commercialscore','negativeregularscore', 'negativescore', 'positivescore',
                              'score', 'grade'], 
                             axis=1)
covid_stats=covid_stats[covid_stats.state=='NY']
covid_stats=covid_stats.drop(['state'], axis=1)
covid_stats=covid_stats.sort_values(by=['date'])
covid_stats['day_count'] = list(range(1,len(covid_stats)+1))

In [4]:
covid_stats.head()

,date,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,onventilatorcumulative,...,deathconfirmed,deathprobable,positiveincrease,negativeincrease,total,totaltestresults,totaltestresultsincrease,deathincrease,hospitalizedincrease,day_count
8138,2020-03-04,6.0,48.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0,0,78,54,0,0,0,1
8120,2020-03-05,22.0,76.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,16,28,122,98,44,0,0,2
8093,2020-03-06,33.0,92.0,236.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11,16,361,125,27,0,0,3
8053,2020-03-07,76.0,92.0,236.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,43,0,404,168,43,0,0,4
8003,2020-03-08,105.0,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,29,0,197,197,29,0,0,5


In [8]:
OPTIM_DAYS = 21
population=19453556

In [9]:
def eval_model_const(params, covid_stats, population, return_solution=False, forecast_days=0):
    R_0, t_hosp, t_crit, m, c, f = params
    N = population
    n_infected = covid_stats['positiveincrease'].iloc[0]
    max_days = len(covid_stats) + forecast_days
    initial_state = [(N - n_infected)/ N, 0, n_infected / N, 0, 0, 0, 0]
    args = (R_0, 5.6, 2.9, t_hosp, t_crit, m, c, f)
               
    sol = solve_ivp(SEIR_HCD_model, [0, max_days], initial_state, args=args, t_eval=np.arange(0, max_days))
    
    sus, exp, inf, rec, hosp, crit, deaths = sol.y
    
    y_pred_cases = np.clip(inf + rec + hosp + crit + deaths, 0, np.inf) * population
    y_true_cases = covid_stats['positive'].values
    y_true_fat = covid_stats['deathconfirmed'].values
    
    optim_days = min(OPTIM_DAYS, len(data))  # Days to optimise for
    weights = 1 / np.arange(1, optim_days+1)[::-1]  # Recent data is more heavily weighted
    msle_cases = mean_squared_log_error(y_true_cases[-optim_days:],  weights)
    msle_fat = mean_squared_log_error(y_true_fat[-optim_days:], weights)
    
    msle_final = np.mean([msle_cases, msle_fat])
    
    if return_solution:
        return msle_final, sol
    else:
        return msle_final

C:\Users\yashv\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:795: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
